<a href="https://colab.research.google.com/github/nedokormysh/GB_NLP_intro/blob/lesson12/NLP_intro_hw_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [34]:
# Функция чтения данных

import json
import random

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [35]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

In [36]:
import pandas as pd

dataset = pd.DataFrame(test_records)
dataset.head(3)

,url,text,title,summary,date
0,https://www.gazeta.ru/science/2020/02/14_a_129...,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",2020-02-14 16:39:11
1,https://www.gazeta.ru/social/2020/02/28/129806...,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,2020-02-28 10:44:13
2,https://www.gazeta.ru/politics/2019/11/07_a_12...,7 ноября в Белоруссии прошли выборы членов сов...,Вспомнить СССР: как Лукашенко провел выборы,В Белоруссии в день годовщины Октябрьской рево...,2019-11-07 19:55:08


In [37]:
dataset = dataset.drop(['url', 'date'], axis=1)

In [38]:
# оченка качества

!pip install rouge -q
!pip install razdel -q

Lead-3

In [39]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [40]:
import razdel

def calc_lead_n_score(records, summary_col = 'summary', n=3, lower=True, nrows=1000):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if i >= nrows:
            break

        summary = record[summary_col]
        summary = summary if not lower else summary.lower()
        references.append(summary)

        text = record["text"]
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

calc_lead_n_score(test_records, n=1)

Count: 1000
Ref: телеканал «спас» запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер. организаторы отметили, что это беспрецедентный подобный проект на телевидении. участникам шоу будет, где поработать — в монастыре работают свечной, молочный и столярный цеха, есть коровник, конюшня, пасека.
Hyp: православный телеканал «спас», учредителем которого является московская патриархия, запускает реалити-шоу «остров», участникам которого предстоит месяц жить и работать в нило-столобенской пустыни на озере селигер в тверской области.
BLEU:  0.19177311186434495
ROUGE:  {'rouge-1': {'r': 0.37762764047433917, 'p': 0.22208274285774904, 'f': 0.23804097238957525}, 'rouge-2': {'r': 0.15833772153385062, 'p': 0.09647636782929753, 'f': 0.10027796832321115}, 'rouge-l': {'r': 0.34937017731940756, 'p': 0.2022959168891477, 'f': 0.21799992093276083}}


Модель MBARTRuSumGazeta

In [41]:
!pip install transformers sentencepiece -q

In [42]:
# Загрузка модели 

from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [43]:
# рассмотрим первые 10

dataset = dataset[:10]

In [44]:
# Генерация краткого содержания для всего датасета

data = dataset['text']

summaries = []

for row in data:
    input_ids = tokenizer(
        [row],
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    summaries.append(summary)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [45]:
dataset['pred_summary'] = summaries

In [46]:
dataset.sample(5)

,text,title,summary,pred_summary
6,Выступление главы российской делегации Петра Т...,"«Смотрите, что у вас происходит»: как прервали...",Активист сорвал выступление главы делегации РФ...,Выступление главы российской делегации Петра Т...
9,Госдума одобрила в первом чтении внесенные пре...,Дождались: работающим пенсионерам восстановят ...,Работающие пенсионеры дождались индексации. Эт...,Госдума одобрила в первом чтении внесенные пре...
1,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,В Кремле прошла премьера новогоднего шоу «Танц...
8,Российский актер и театральный педагог Виталий...,Умер актер сериала «Улицы разбитых фонарей»,Актер сериала «Улицы разбитых фонарей» Виталий...,Российский актер и театральный педагог Виталий...
4,Депутат Верховной рады от партии «Слуга народа...,«Поддерживают Россию»: почему Киев не платит п...,Украина не должна выплачивать пенсии жителям Д...,Депутат Верховной рады от партии «Слуга народа...


In [47]:
predictions = dataset['pred_summary']
references = dataset['summary']

In [49]:
calc_scores(references.values, predictions.values, metric="all")

Count: 10
Ref: Работающие пенсионеры дождались индексации. Это следует из уже одобренных в первом чтении поправок в Конституцию. Так что работающие пенсионеры смогут получить замороженную с 2016 года прибавку к пенсии. В России их насчитывается более девяти миллионов. Эксперты называют это восстановлением справедливости.
Hyp: Госдума одобрила в первом чтении внесенные президентом РФ Владимиром Путиным поправки в Конституцию. Одно из них — индексация пенсий работающим пенсионерам. По мнению экспертов, это означает, что государство готово выполнять свои обязательства и в тяжелые для пенсионной системы годы.
BLEU:  0.4070334044292795
ROUGE:  {'rouge-1': {'r': 0.24809582503610272, 'p': 0.22031645088748153, 'f': 0.2307363685067823}, 'rouge-2': {'r': 0.09920773138094865, 'p': 0.0885193682896723, 'f': 0.09181747688260844}, 'rouge-l': {'r': 0.23811047705075478, 'p': 0.21193892563495628, 'f': 0.22183650691249807}}


In [ ]:
# import torch
# # https://huggingface.co/cointegrated/rut5-base-absum

# model.eval();

# def summarize(
#     text, n_words=None, compression=None,
#     max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
#     **kwargs
# ):
#     """
#     Summarize the text
#     The following parameters are mutually exclusive:
#     - n_words (int) is an approximate number of words to generate.
#     - compression (float) is an approximate length ratio of summary and original text.
#     """
#     if n_words:
#         text = '[{}] '.format(n_words) + text
#     elif compression:
#         text = '[{0:.1g}] '.format(compression) + text
#     x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
#     with torch.inference_mode():
#         out = model.generate(
#             **x, 
#             max_length=max_length, num_beams=num_beams, 
#             do_sample=do_sample, repetition_penalty=repetition_penalty, 
#             **kwargs
#         )
#     return tokenizer.decode(out[0], skip_special_tokens=True)